Getting files (from pico do couto radar and INMET). First, we will compare just the year 2021: 

In [135]:
pc_path = "../data/PC-probative-analysis/RADAR-PC/2021/"
copa_path = "../data/PC_probative_analysis/inmetcopa.CSV"

Building a feature (csv) from images from Pico Do couto related to a region of interest. In this case, "FORTE COPACABANA".
We know the coordinates: Lat: -22.98833333, Long: -43.19055555

In [1]:
import pandas as pd 
import cv2 
import os 

Finding the points of Forte Copacabana in the picture (250x250), knowing the reference of Pico do Couto: Lat: -22.464278, Long: -43.297476

{"id":25,"nome":"Radar - Pico do Couto\/RJ","localidade":"pc","raio":250,"lat_center":"-22.464278","lon_center":"-43.297476","lon_min":"-45.7583","lon_max":"-40.8262","lat_min":"-24.701","lat_max":"-20.2013"}

Ranges of Lat and Long:

In [82]:
lon_min = -45.7583
lon_max = -40.8262
lat_min = -24.701
lat_max = -20.2013

rate_lon = lon_max - lon_min
rate_lat = lat_max - lat_min

In [109]:
#razão:
latitude_pc = 4.4997
longitude_pc = 4.9321
        
def pontos(x,y):
    
          long_pc, lat_pc = -22.464278, -43.297476

          dist_x =  (x - long_pc) 
          dist_y = (y - lat_pc)

          x1 = 125 - (dist_x * 250 / rate_lon)
          y1 = 125 + (dist_y * 250 / rate_lat)

          x1, y1 = round(x1), round(y1)

          return y1,x1

y1, x1 = pontos(-22.98833333, -43.19055555)

In [128]:
df_pc = pd.DataFrame()

for mouth in os.listdir(pc_path):
    mouth_path = os.path.join(pc_path, mouth)
    
    for day in os.listdir(mouth_path):
        day_path = os.path.join(mouth_path, day)
        
        for picture in os.listdir(day_path):
            file = os.path.join(day_path, picture)

            name_picture = str.replace(picture,".png","")
            image = cv2.imread(file)

            pixel = image[y1, x1]

            B = pixel[0]
            G = pixel[1]
            R = pixel[2]

            dic = {"R": [R], "G": [G],"B": [B]}
            
            d_colors = pd.DataFrame(dic)
            d_colors["timestamp"] = name_picture
            d_colors["R"] = R
            d_colors["G"] = G
            d_colors["B"] = B

            df_pc = pd.concat([df_pc,d_colors])

In [131]:
df_pc

,R,G,B,timestamp
0,0,0,0,2021-08-15--211026
0,0,0,0,2021-08-15--095026
0,0,0,0,2021-08-15--231025
0,0,0,0,2021-08-15--155026
0,0,0,0,2021-08-15--071025
...,...,...,...,...
0,0,0,0,2021-04-04--173025
0,0,0,0,2021-04-04--115026
0,0,0,0,2021-04-04--055026
0,0,0,0,2021-04-04--043026


In [136]:
df_copa = pd.read_csv(copa_path)

FileNotFoundError: [Errno 2] No such file or directory: '../data/PC_probative_analysis/inmetcopa.CSV'